In [1081]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import and read charity_data.csv
import pandas as pd 
application_df = pd.read_csv('../Resources/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# 3. Preprocess the data

In [1082]:
# Drop non-beneficial ID columns
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

In [1083]:
# Determine the number of unique values in each column
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [1084]:
# APPLICATION_TYPE value counts for binning
application_values = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_application = application_values[application_values < 1000].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, 'Other')

In [1085]:
# CLASSIFICATION value counts for binning
classification_values = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_class = classification_values[classification_values < 1000].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls, 'Other')

In [1086]:
# ASK_AMT bins
ask_values_0 = application_df['ASK_AMT'][application_df['ASK_AMT'] == 5000]
ask_values_1 = application_df['ASK_AMT'][(application_df['ASK_AMT'] > 5000)]

# Replace in dataframe
for amt in ask_values_0:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '5000')
for amt in ask_values_1:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '>5000')

In [1087]:
# AFFILIATION value counts for binning
affiliation_values = application_df['AFFILIATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_affiliation = affiliation_values[affiliation_values < 1000].index.tolist()

# Replace in dataframe
for aff in replace_affiliation:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(aff, 'Other')

In [1089]:
# ORGANIZATION value counts for binning
organization_values = application_df['ORGANIZATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_organization = organization_values[organization_values < 1000].index.tolist()

# Replace in dataframe
for org in replace_organization:
    application_df.ORGANIZATION = application_df.ORGANIZATION.replace(org, 'Other')

In [1090]:
# Generate the categorical variable list
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [1091]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

In [1092]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1)

In [1093]:
application_df.columns

Index(['IS_SUCCESSFUL', 'APPLICATION_TYPE_Other', 'APPLICATION_TYPE_T19',
       'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5',
       'APPLICATION_TYPE_T6', 'AFFILIATION_CompanySponsored',
       'AFFILIATION_Independent', 'AFFILIATION_Other', 'CLASSIFICATION_C1000',
       'CLASSIFICATION_C1200', 'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100',
       'CLASSIFICATION_C3000', 'CLASSIFICATION_Other',
       'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other',
       'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Other', 'ORGANIZATION_Trust',
       'INCOME_AMT_0', 'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
       'ASK_AMT_5000', 'ASK_AMT_>5000'],
      dtype='object')

In [1094]:
# Split the preprocessed data into features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [1095]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# 4. Design a neural network model

In [1096]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_190 (Dense)            (None, 80)                2800      
_________________________________________________________________
dense_191 (Dense)            (None, 30)                2430      
_________________________________________________________________
dense_192 (Dense)            (None, 10)                310       
_________________________________________________________________
dense_193 (Dense)            (None, 1)                 11        
Total params: 5,551
Trainable params: 5,551
Non-trainable params: 0
_________________________________________________________________


In [1097]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 5. Create a callback

In [1098]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('optimized_checkpoints/', exist_ok=True)
checkpoint_path = 'optimized_checkpoints/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [1099]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 976us/step - loss: 0.5998 - accuracy: 0.6964
Epoch 2/100
804/804 [==============================] - 1s 962us/step - loss: 0.5745 - accuracy: 0.7199
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5679 - accuracy: 0.7247
Epoch 4/100
804/804 [==============================] - 1s 992us/step - loss: 0.5645 - accuracy: 0.7275
Epoch 5/100
776/804 [===========================>..] - ETA: 0s - loss: 0.5635 - accuracy: 0.7256
Epoch 00005: saving model to optimized_checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5630 - accuracy: 0.7261
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5608 - accuracy: 0.7270
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5595 - accuracy: 0.7289
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5600 - accuracy: 0.7301
Epoch 9/100
804/804 [=============

804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7350
Epoch 64/100
804/804 [==============================] - 1s 964us/step - loss: 0.5485 - accuracy: 0.7347
Epoch 65/100
530/804 [==================>...........] - ETA: 0s - loss: 0.5494 - accuracy: 0.7335
Epoch 00065: saving model to optimized_checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7347
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7355
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7357
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7356
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7348
Epoch 70/100
494/804 [=================>............] - ETA: 0s - loss: 0.5458 - accuracy: 0.7374
Epoch 00070: saving model to optimized_checkpoi

In [1100]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.8742 - accuracy: 0.7058
Loss: 0.8742496371269226, Accuracy: 0.7057725787162781


# 6. Export results

In [ ]:
# Export the model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')